In [0]:
%fs
ls dbfs:/mnt/Anac

In [0]:
%fs
ls dbfs:/mnt/Anac/Bronze

Agora o Databricks consegue acessar ao Azure como se os arquivos tivesem sido importados nele mesmo.

In [0]:
df = spark.read.json('dbfs:/mnt/Anac/Bronze/V_OCORRENCIA_AMPLA.json')
display(df)

In [0]:
lista_colunas = [
    "Aerodromo_de_Destino", "Aerodromo_de_Origem", "CLS", "Categoria_da_Aeronave", "Classificacao_da_Ocorrência", "Danos_a_Aeronave", "Descricao_do_Tipo", "Fase_da_Operacao",
    "Historico", "ICAO", "Matricula", "Modelo", "Municipio", "Nome_do_Fabricante",
    "Operacao", "Operador", "Operador_Padronizado", "PMD", "Regiao", "Tipo_ICAO", "Tipo_de_Aerodromo", "Tipo_de_Ocorrencia", "UF"
]

for ajuste in lista_colunas:
    df = df.fillna('Sem Registro', subset=[ajuste])

In [0]:
lista_colunas_int = [
    'Ilesos_Passageiros', 'Ilesos_Tripulantes', 'Lesoes_Desconhecidas_Tripulantes', 'Lesoes_Desconhecidas_Terceiros', 'Lesoes_Desconhecidas_Passageiros', 'Lesoes_Fatais_Passageiros',
    'Lesoes_Fatais_Terceiros', 'Lesoes_Fatais_Tripulantes', 'Lesoes_Graves_Passageiros', 'Lesoes_Graves_Terceiros', 'Lesoes_Graves_Tripulantes', 'Lesoes_Leves_Passageiros', 'Lesoes_Leves_Terceiros', 'Lesoes_Leves_Tripulantes', 'Numero_de_Assentos', 'Numero_da_Ocorrencia',
    'Numero_da_Ficha'
]
# display(
#     df\
#         .withColumn('Lesoes_Fatais_Terceiros', df['Lesoes_Fatais_Terceiros'].cast('int'))\
#         .fillna(0, subset='Lesoes_Fatais_Terceiros')
#         .select('Lesoes_Fatais_Terceiros')
# )
for coluna in lista_colunas_int:
    df = df\
        .withColumn(coluna, df[coluna].cast('int'))\
        .fillna(0, subset=coluna)


In [0]:


df = df.withColumn('Data_da_Ocorrencia', df['Data_da_Ocorrencia'].cast('date'))
display(df.select('Data_da_Ocorrencia'))

In [0]:
from pyspark.sql.functions import regexp_replace, col

df = df\
    .withColumn(
        'Latitude', 
        regexp_replace('Latitude', ',', '.').cast('decimal(8,6)')
    )


df = df\
    .withColumn(
        'Longitude', 
        regexp_replace('Longitude', ',', '.').cast('decimal(9,6)')
    )

display(df.select('Latitude', 'Longitude'))


In [0]:
Não sei oque fazer com a coluna "Hora_da_Ocorrência"

In [0]:
from pyspark.sql.functions import when

df = df\
    .withColumn("PSSO", \
        when(df.PSSO == 'verdadeiro', True).when(df.PSSO == 'falso', False)
    )

In [0]:
df.write\
    .format("parquet")\
    .mode("overwrite")\
    .option("compression", "gzip")\
    .option("header", "true")\
    .save("dbfs:/mnt/Anac/Silver/anac_silver.parquet")

In [0]:
%fs ls dbfs:/mnt/Anac/Silver

In [0]:
display(spark.read\
    .format("parquet")\
    .option("header", True)\
    .option("inferSchema", True)\
    .option("compression", "gzip")
    .load("dbfs:/mnt/Anac/Silver/anac_silver.parquet"))

In [0]:
print("A")